In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.linear_model import LinearRegression

In [3]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [4]:
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리

In [5]:
# 데이터 불러오기
babyox = pd.read_csv('/content/drive/MyDrive/2023_동계방학_빅데이터 활용 미래사회 문제 예측/활용 데이터/자녀 계획 여부/자녀_계획_여부_격년__20240112004157.csv', encoding='cp949')
babyox.head(3)

,구분별(1),구분별(2),2022,2022.1,2022.2,2022.3
0,구분별(1),구분별(2),여성,여성,남성,남성
1,구분별(1),구분별(2),있다,없다,있다,없다
2,전체,소계,11.1,88.9,13.6,86.4


In [6]:
# 열 이름 변경
babyox.columns = ['구분별(1)', '구분별(2)', '자녀계획 유_여성', '자녀계획 무_여성', '자녀계획 유_남성', '자녀계획 무_남성']

re_babyox = babyox.drop([0], axis = 0)
re_babyox.head(3)

,구분별(1),구분별(2),자녀계획 유_여성,자녀계획 무_여성,자녀계획 유_남성,자녀계획 무_남성
1,구분별(1),구분별(2),있다,없다,있다,없다
2,전체,소계,11.1,88.9,13.6,86.4
3,규모,100~199인,11.2,88.8,15.4,84.6


In [8]:
# 여성 데이터만 추출
babyox_woman = re_babyox[['구분별(1)', '구분별(2)'] + [col for col in re_babyox.columns if '여성' in col]]
babyox_woman['성별'] = '여성'
new_columns = {'구분별(1)' : '구분별(1)', '구분별(2)':'구분별(2)', '자녀계획 유_여성':'자녀계획 유', '자녀계획 무_여성':'자녀계획 무'}

babyox_woman = babyox_woman.rename(columns=new_columns)
babyox_woman_drop = babyox_woman.dropna(axis=0)
babyox_woman_drop.head(3)

<ipython-input-8-07819921af87>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  babyox_woman['성별'] = '여성'


,구분별(1),구분별(2),자녀계획 유,자녀계획 무,성별
1,구분별(1),구분별(2),있다,없다,여성
2,전체,소계,11.1,88.9,여성
3,규모,100~199인,11.2,88.8,여성


In [12]:
# 필요없는 행 삭제
last_babyox = babyox_woman.drop([1], axis = 0)
last_babyox.head(3)

,구분별(1),구분별(2),자녀계획 유,자녀계획 무,성별
2,전체,소계,11.1,88.9,여성
3,규모,100~199인,11.2,88.8,여성
4,규모,200~299인,5.7,94.3,여성


In [9]:
# 남성 데이터만 추출
babyox_man = re_babyox[['구분별(1)', '구분별(2)'] + [col for col in re_babyox.columns if '남성' in col]]
babyox_man['성별'] = '남성'
new_columns = {'구분별(1)' : '구분별(1)', '구분별(2)':'구분별(2)', '자녀계획 유_남성':'자녀계획 유', '자녀계획 무_남성':'자녀계획 무'}

babyox_man = babyox_man.rename(columns=new_columns)
babyox_man_drop = babyox_man.dropna(axis=0)
babyox_man_drop.head(3)

<ipython-input-9-6ff38f4cb058>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  babyox_man['성별'] = '남성'


,구분별(1),구분별(2),자녀계획 유,자녀계획 무,성별
1,구분별(1),구분별(2),있다,없다,남성
2,전체,소계,13.6,86.4,남성
3,규모,100~199인,15.4,84.6,남성


In [13]:
# 필요없는 행 삭제
last_babyox = babyox_man_drop.drop([1], axis = 0)
last_babyox.head(3)

,구분별(1),구분별(2),자녀계획 유,자녀계획 무,성별
2,전체,소계,13.6,86.4,남성
3,규모,100~199인,15.4,84.6,남성
4,규모,200~299인,17.8,82.2,남성


In [15]:
# 데이터 통합
new_babyox = pd.concat([last_babyox, last_babyox])
new_babyox.head(100)

,구분별(1),구분별(2),자녀계획 유,자녀계획 무,성별
2,전체,소계,13.6,86.4,남성
3,규모,100~199인,15.4,84.6,남성
4,규모,200~299인,17.8,82.2,남성
5,규모,300~499인,9.3,90.7,남성
6,규모,500인 이상,12.9,87.1,남성
7,직급,과장급(이하),21.2,78.8,남성
8,직급,차장급,7.7,92.3,남성
9,직급,부장급,5.5,94.5,남성
10,직급,임원급,-,100.0,남성
11,업종,농업 광업 제조업,9.2,90.8,남성
